In [140]:
import pandas as pd
import numpy as np

slots = pd.read_csv('./march-machine-learning-mania-2024/slots_2024.csv')
results = pd.read_csv('./labels/submission01.csv')
refs = pd.read_csv('./labels/reference.csv').sort_values(by='Slot')
results = results[results['Tournament']=='M']

In [141]:
# Predict round-by-round from R1 to R6 (Championship)

def random_prediction():
    Round = ['R'+str(i) for i in range(1, 7)]
    result_table = slots.copy().reset_index(drop=True)

    random_preds = []
    for i in range(len(Round)):
        current_round_df = result_table[result_table['DayCategory']==Round[i]]

        # Add teamid to df
        #current_round_df = current_round_df.merge(mseeds2024, left_on=['Season', 'StrongSeed'], right_on=['Season', 'Seed'], how='left')
        #current_round_df = current_round_df.merge(mseeds2024, left_on=['Season', 'WeakSeed'], right_on=['Season', 'Seed'], how='left')
        #current_round_df = current_round_df.drop(columns=['Seed_x', 'Seed_y']).rename(columns={'StrongSeed': 'WSeed', 'WeakSeed': 'LSeed', 'TeamID_x': 'WTeamID', 'TeamID_y': 'LTeamID'})

        random_pred = np.random.randint(2, size=len(current_round_df))
        
        pred_df = pd.DataFrame({
            'Slot': current_round_df['Slot'],
            'Pred_Winner': random_pred})
        
        # Write the prediction results onto result_table and prediction dfs
        for i, row in pred_df.iterrows():
            if row['Pred_Winner'] == 1:
                winner = result_table.loc[result_table['Slot']==row['Slot'], 'StrongSeed'].values[0]
                
                if row['Slot'] in result_table['StrongSeed'].values:
                    result_table.loc[result_table['StrongSeed']==row['Slot'], 'StrongSeed'] = winner
                else:
                    result_table.loc[result_table['WeakSeed']==row['Slot'], 'WeakSeed'] = winner
            else:
                winner = result_table.loc[result_table['Slot']==row['Slot'], 'WeakSeed'].values[0]
                
                if row['Slot'] in result_table['StrongSeed'].values:
                    result_table.loc[result_table['StrongSeed']==row['Slot'], 'StrongSeed'] = winner
                else:
                    result_table.loc[result_table['WeakSeed']==row['Slot'], 'WeakSeed'] = winner
        
            random_preds.append(winner)    

        if i > len(Round) - 2:
            continue 
        
        for j, row in result_table[result_table['DayCategory']==Round[i+1]].iterrows():
            strong = result_table.loc[j, 'StrongSeed']
            weak = result_table.loc[j, 'WeakSeed']
            if int(strong[1:3]) > int(weak[1:3]):
                result_table.loc[j, 'StrongSeed'], result_table.loc[j, 'WeakSeed'] = weak, strong

    random_preds = np.array(random_preds)

    return random_preds

In [146]:
target_round = ['R1']
N = 3000 # Num of simulation

num_corrects = []
for i in range(N):
    mask = [r in target_round for r in results['Slot'].str.slice(0,2)]

    random_preds = random_prediction()
    correct = (refs[mask]['Reference'].values==random_preds[mask]).sum()

    num_corrects.append(correct)

avg_correct = np.array(num_corrects).mean()
total = len(refs[mask])
acc = avg_correct / total
print(f'Accuracy of random prediction: {round(acc, 3)}')

Accuracy of random prediction: 0.501
